In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import madmom
import mir_eval
from modules.analysis_funcs import aubio_peakpicker_do, aubio_postprocessing
from analyze_detection import evaluate

from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt


In [9]:
from datasets import Dataset
ds0 = Dataset("initslurtest")
ds1 = Dataset("slurtest_add_1")

audio_fnames = ds0.get_audio_paths() + ds1.get_audio_paths()
label_fnames = ds0.get_annotation_paths() + ds1.get_annotation_paths()
audios = [madmom.audio.signal.load_wave_file(filename)[0] for filename in audio_fnames]
sample_rates = [madmom.audio.signal.load_wave_file(filename)[1] for filename in audio_fnames]
onset_schedules = [np.loadtxt(label_fname, usecols=0) for label_fname in label_fnames]

/usr/local/lib/python3.9/site-packages/madmom/audio/signal.py:457: UserWarning: Deprecated as of version 0.16. Please use madmom.io.audio.load_wave_file instead. Will be removed in version 0.18.
  warnings.warn('Deprecated as of version 0.16. Please use madmom.io.audio.'
/usr/local/lib/python3.9/site-packages/madmom/io/audio.py:493: WavFileWarning: Chunk (non-data) not understood, skipping it.
  file_sample_rate, signal = wavfile.read(filename, mmap=True)


In [15]:
sigg = madmom.audio.signal.SignalProcessor(num_channels=1)
frame = madmom.audio.signal.FramedSignalProcessor(frame_size=512, hop_size=441)
stft = madmom.audio.stft.STFTProcessor()
proc = madmom.processors.SequentialProcessor([sigg, frame, stft])

In [11]:
SHIFT = 5
THRES = 0.15
TOL = 0.025
HOP = 441



In [12]:
THRESS = [0.001,0.01,0.05,0.1,0.15,0.2,0.5,0.9]
cds = np.zeros((len(THRESS), len(audio_fnames)))
fps = np.zeros((len(THRESS), len(audio_fnames)))
fns = np.zeros((len(THRESS), len(audio_fnames)))
tns = np.zeros((len(THRESS), len(audio_fnames)))

In [17]:
positives = np.zeros(len(audios))
negatives = np.zeros(len(audios))
for a, onsets in enumerate(onset_schedules):
    positives[a] = np.sum(onsets)
    negatives[a] = len(onsets) - np.sum(onsets)
    

In [16]:
for a, (fname, audio, onsets, sr) in enumerate(zip(audio_fnames, audios, onset_schedules, sample_rates)):
    
    for t, THRES in enumerate(THRESS):
        spect = proc(fname)
        hfc = madmom.features.onsets.high_frequency_content(np.abs(spect))
        onehot, out = aubio_peakpicker_do(hfc, threshold=THRES)
        peaks = (np.where(onehot==1)[0]-SHIFT)*HOP/sr
        [CD,FN,FP,doubles,merged] = evaluate(onsets, peaks, tol_sec=TOL)
        cds[t, a] = CD
        fps[t, a] = FP
        fns[t, a] = FN
        tns[t, a] = (len(onehot)-onehot.sum()) - FN
    print(fname)
    print(cds[:,a]/(cds[:,a]+0.5*fps[:,a]+fns[:,a]))


datasets/initslurtest_vn/initslurtest_vn_wav/slurtest01.wav
[0.39616613 0.41059603 0.52765957 0.62564103 0.73493976 0.84507042
 0.625      0.06299213]
datasets/initslurtest_vn/initslurtest_vn_wav/slurtest02.wav
[0.38538206 0.40277778 0.5        0.60103627 0.69090909 0.72368421
 0.64516129 0.17886179]
datasets/initslurtest_vn/initslurtest_vn_wav/slurtest03.wav
[0.40410959 0.41549296 0.5        0.61780105 0.69005848 0.71698113
 0.56923077 0.12903226]
datasets/initslurtest_vn/initslurtest_vn_wav/slurtest04.wav
[0.52363636 0.54135338 0.60759494 0.68899522 0.73298429 0.75138122
 0.5        0.07894737]
datasets/initslurtest_vn/initslurtest_vn_wav/slurtest05.wav
[0.33616299 0.35357143 0.42888403 0.545961   0.6        0.67407407
 0.71028037 0.25490196]
datasets/initslurtest_vn/initslurtest_vn_wav/slurtest06.wav
[0.65380906 0.6627451  0.69665272 0.71668533 0.71058824 0.68459658
 0.42758621 0.11931818]
datasets/initslurtest_vn/initslurtest_vn_wav/slurtest07.wav
[0.64068441 0.64859632 0.68835257 

/usr/local/lib/python3.9/site-packages/madmom/io/audio.py:493: WavFileWarning: Chunk (non-data) not understood, skipping it.
  file_sample_rate, signal = wavfile.read(filename, mmap=True)


datasets/initslurtest_vn/initslurtest_vn_wav/slurtest13.wav
[0.20754717 0.22222222 0.28476821 0.3943662  0.50909091 0.6
 0.63157895 0.28571429]
datasets/initslurtest_vn/initslurtest_vn_wav/slurtest14.wav
[0.20773931 0.22077922 0.28571429 0.38059701 0.43518519 0.52380952
 0.45217391 0.18181818]
datasets/initslurtest_vn/initslurtest_vn_wav/slurtest15.wav
[0.35211268 0.36297641 0.43162393 0.51546392 0.59327217 0.66896552
 0.5520362  0.27751196]
datasets/initslurtest_vn/initslurtest_vn_wav/slurtest16.wav
[0.32611833 0.3419062  0.40794224 0.48660714 0.55263158 0.59347181
 0.60557769 0.19574468]
datasets/initslurtest_vn/initslurtest_vn_wav/slurtest17.wav
[0.1681128  0.17775229 0.22173596 0.286      0.33628319 0.3788122
 0.26330532 0.06626506]
datasets/initslurtest_vn/initslurtest_vn_wav/slurtest18.wav
[0.05650858 0.05970149 0.07738095 0.11899314 0.17921147 0.22335025
 0.02777778 0.        ]
datasets/initslurtest_vn/initslurtest_vn_wav/slurtest19.wav
[0.38709677 0.4020202  0.47815534 0.582582

In [4]:
for win_size in [256, 512, 1024]:
    cds = np.load("results/hfc-eval-220422/hfc_AB_w"+ str(win_size)+"_tuning_cds.npy")
    fns = np.load("results/hfc-eval-220422/hfc_AB_w"+ str(win_size)+"_tuning_fns.npy")
    fps = np.load("results/hfc-eval-220422/hfc_AB_w"+ str(win_size)+"_tuning_fps.npy")
    tpr = cds.sum(1)/(cds.sum(1)+fns.sum(1))
    fpr = fps.sum(1)/(fps.sum(1)+tns.sum(1))
    fscores = cds.sum(1)/(cds.sum(1)+0.5*fps.sum(1)+0.5*fns.sum(1))
    plt.plot(fpr, tpr, '-o')
plt.show()


NameError: name 'tns' is not defined

In [5]:
fscores

NameError: name 'tns' is not defined

In [14]:
np.save(file="results/hfc-eval-220422/hfc_AB_w512_tuning_tns.npy", arr=fns)